In [82]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np



# empty data frame with columns

In [137]:
apple_forum = pd.DataFrame({'title' : [], 'date': [], 'user':[], 'nb_views':[], 'nb_replies':[], 'link':[]})

In [147]:
html = requests.get("https://discussions.apple.com/community/iphone/iphone_hardware").text
soup = BeautifulSoup(html, 'html.parser')
bs_object = soup.find_all('div', attrs={"class":u"discussion-description"})

## find all the elements in the page corresponding to a thread

In [148]:
for s,index in zip(bs_object,range(len(bs_object))): 
    #links to threads
    apple_forum.loc[index,'link'] = s.find('div', attrs={"class":u"discussion-title"}).find('a').get('href')
    
    #question text
    apple_forum.loc[index,'title'] = s.find('div', attrs={"class":u"discussion-title"}).find('a') \
    .find('span', attrs={"class":u"discussion-description-inner-responsive u-hide--desktop"}).get_text().strip()
    
    #view number
    apple_forum.loc[index,'nb_views'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
          .find('div', attrs={"class":u"cell discussion-views u-hide--small"}).get_text().strip()
    
    #user name
    apple_forum.loc[index,'user'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
    .find('div', attrs={"class":u"cell discussion-name u-hide--small"}).get_text().strip()
    
    #latest reply and user replying
    apple_forum.loc[index,'date'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
    .find('div', attrs={"class":u"cell align-metadata"}).get_text().strip()
    
    #latest reply and user replying
    apple_forum.loc[index,'nb_replies'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
    .find_all('div', attrs={"class":"cell align-metadata"})[1].get_text().strip()
    

In [149]:
apple_forum

,date,link,nb_replies,nb_views,title,user
0,latest reply 15 minutes ago\n ...,/thread/8232044,1 reply,8 views,iPhone 6S Fire Issue,asked by bardeb15
1,latest reply 23 minutes ago\n ...,/thread/8230917,2 replies,42 views,Memory problems,asked by dontthink
2,latest reply 24 hours ago\n ...,/thread/8230224,1 reply,22 views,Speaker doesn’t work,asked by Edvardas98
3,latest reply 41 minutes ago\n ...,/thread/8230916,5 replies,29 views,iPhone x Bottom Speaker,asked by PeeGee76
4,latest reply 45 minutes ago\n ...,/thread/8204632,45 replies,"7,832 views",battery life iOS 11.2.1,asked by ngunter
5,latest reply 49 minutes ago\n ...,/thread/8155725,10 replies,676 views,Voice recording bug on iPhone X?,asked by Mac Ed
6,49 minutes ago,/thread/8231976,0 replies,4 views,anyone else's phone shutting down from full ba...,asked by mercedeskf
7,latest reply 1 hour ago\n ...,/thread/8231803,2 replies,16 views,How to?,asked by 1scssimmons
8,latest reply 1 hour ago\n ...,/thread/8189731,3 replies,99 views,Apple mail-in repair return unfixed iphone,asked by Summerplum
9,latest reply 5 hours ago\n ...,/thread/8231464,2 replies,23 views,I need immediate help regarding a lost iPhone....,asked by Baseballmom27
